In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json
import dataset
import credentials
import sqlite3
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pickle
from Naive_bayes_model import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
f = open('nb.pickle', 'rb')
NBayesClassifier = pickle.load(f)
f.close()

In [3]:
conn = sqlite3.connect('tweet2.db')
c = conn.cursor()

In [4]:
def pred(data):
    
    tweetProcessor = PreProcessTweets()
    preprocessedTrainingSet = tweetProcessor.processTweets(data)
    
    word_features = buildVocabulary(preprocessedTrainingSet)
#     print("\n",word_features,"\n")
#     buildVocabulary(preprocessedTrainingSet)
    trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)
    
    print(preprocessedTrainingSet[0][0])
    
    NB = NBayesClassifier.classify(extract_features(preprocessedTrainingSet[0][0],word_features))
    
    return NB
    

In [5]:
class TwitterStreamer():
    
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
        auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
        
        stream = Stream(auth, listener)
        
        stream.filter(track=['accident','fire','crash','ablaze','die','destroy'])
        
 
class StdOutListener(StreamListener):
    
    def on_status(self, status):
        print(status.text)

    def on_error(self, status):
        print(status)

    def on_data(self, data):
        all_data             = json.loads(data)
        created_at           = all_data['created_at']
        favorite_count       = all_data['favorite_count']
        favorited            = all_data['favorited']
        filter_level         = all_data['filter_level']
        lang                 = all_data['lang']
        retweet_count        = all_data['retweet_count']
        retweeted            = all_data['retweeted']
        source               = all_data['source']
        text                 = all_data['text']
        truncated            = all_data['truncated']
        user_created_at      = all_data['user']['created_at']
        user_followers_count = all_data['user']['followers_count']
        user_location        = all_data['user']['location']
        user_lang            = all_data['user']['lang']
        user_name            = all_data['user']['name']
        user_screen_name     = all_data['user']['screen_name']
        user_time_zone       = all_data['user']['time_zone']
        user_utc_offset      = all_data['user']['utc_offset']
        user_friends_count   = all_data['user']['friends_count']
        geo_enabled          = all_data['user']['geo_enabled']  
        coordinates          = all_data['coordinates']
        geo                  = all_data['geo']
        place                = all_data['place']
        if place is not None:
            place_name       = all_data['place']['name']
        else:
            place_name       = all_data['place']
        
        print("Tweet: "+text+"\n")
        
        data = [{'text': text,'Classfication': ''}]
        
        info = pred(data)
        
        print("\n Class: "+info+"\n")
                         
        c.execute('''INSERT INTO tweets 
        (created_at, favorite_count, favorited, filter_level, lang, 
                         retweet_count, retweeted, source, text, truncated, user_created_at,  
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ) 
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
            (created_at, favorite_count, favorited, filter_level, lang, retweet_count, 
                         retweeted, source, text, truncated, user_created_at, 
                         user_followers_count, user_location, user_lang, user_name, 
                         user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ))
        
        c.execute('''INSERT INTO tweet_class 
        (text, class) 
            VALUES (?,?)''', 
            (text, info))
        
        conn.commit()

In [6]:
if __name__ == '__main__':
    
    hash_tag_list = ['accident','fire','crash','ablaze','die','destroy']
    fetched_tweets_filename = "tweets.json"
    
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

Tweet: @phoenix_de @CDU @jensspahn @BMG_Bund Dass Herr Spahn immer noch nicht wahrgenommen hat, dass er einen großen Beitr… https://t.co/kmbdSUbrnp

['dass', 'herr', 'spahn', 'immer', 'noch', 'nicht', 'wahrgenommen', 'hat', 'dass', 'er', 'einen', 'großen', 'beitr…']

 Class: Not Relevant

Tweet: RT @kimbakit: PM "Too busy" - he was:

Rorting grants
Marketing lies
In Hawaii
Overseas (x20)
Dodging questions
Blaming Labor
Denying clima…

['rt', 'pm', '``', 'busy', "''", 'rorting', 'grants', 'marketing', 'lies', 'hawaii', 'overseas', 'x20', 'dodging', 'questions', 'blaming', 'labor', 'denying', 'clima…']

 Class: Can't Decide

Tweet: @raziel765 Ich bin Angelique die Titten Baroness.......habe Grösse 95 E alles natur pur !!

['ich', 'bin', 'angelique', 'die', 'titten', 'baroness', '...', '...', '.habe', 'grösse', '95', 'e', 'alles', 'natur', 'pur']

 Class: Can't Decide

Tweet: Turkish occupation army targeting the town of Nayrab, with artillery fire

['turkish', 'occupation', 'army', 'targ

Tweet: RT @MonsterclipRSPN: This guy then tweets at me asking when his issue will be fixed.

['rt', 'guy', 'tweets', 'asking', 'issue', 'fixed']

 Class: Can't Decide

Tweet: RT @LRaumfeld: Ein angepasster Wendehals im Kielwasser eines Horstes! https://t.co/5XFDozTfH3

['rt', 'ein', 'angepasster', 'wendehals', 'im', 'kielwasser', 'eines', 'horstes']

 Class: Not Relevant

Tweet: RT @saj_malik007: WOWOWOOWOWhoooo00

I m gonna Die Man 

@JohnCena Posted again ❤️❤️❤️❤️❤️😍😍

#InspirationAsim https://t.co/tLeIrreOAF

['rt', 'wowowoowowhoooo00', 'gon', 'na', 'die', 'man', 'posted', '❤️❤️❤️❤️❤️😍😍', 'inspirationasim']

 Class: Can't Decide

Tweet: @Natascha_Strobl Wann hat eigentlich diese Unkultur des Nicht-Zurücktretens begonnen? 

War das Schüssel nach der A… https://t.co/ag1On2uZB9

['wann', 'hat', 'eigentlich', 'diese', 'unkultur', 'des', 'nicht-zurücktretens', 'begonnen', 'war', 'das', 'schüssel', 'nach', 'der', 'a…']

 Class: Can't Decide

Tweet: While Australia is making millions in th

Tweet: 😂😂😂

['😂😂😂']

 Class: Not Relevant

Tweet: RT @meWernerHauser: @Asparag14679849 Der weisungsgebundene Staatsanwalt wird diese Strafanzeige gegen seinen Dienstherrn im eigenen Interes…

['rt', 'der', 'weisungsgebundene', 'staatsanwalt', 'wird', 'diese', 'strafanzeige', 'gegen', 'seinen', 'dienstherrn', 'im', 'eigenen', 'interes…']

 Class: Can't Decide

Tweet: When you will know better, you will act better!

['know', 'better', 'act', 'better']

 Class: Not Relevant

Tweet: @EickeTarkus @2aeugigerZyklop @KShmurtz @Naturmacht @0pposition24com Seit über 50 Jahren haben wir eine sehr niedri… https://t.co/vM6m0h5VG0

['seit', 'über', '50', 'jahren', 'haben', 'wir', 'eine', 'sehr', 'niedri…']

 Class: Relevant

Tweet: RT @RTLZ: De EU maakt al jaren miljarden euro's over naar zes Balkanlanden die wachten tot ze tot de Europese Unie mogen toetreden
https://…

['rt', 'de', 'eu', 'maakt', 'al', 'jaren', 'miljarden', 'euro', "'s", 'naar', 'zes', 'balkanlanden', 'die', 'wachten', 'tot', 'ze'

Tweet: RT @DalitLiveMatter: Man enters 50-year-old #Dalit woman's house, sets her ablaze in #Maharashtra's #Aurangabad district https://t.co/AnOaV…

['rt', 'man', 'enters', '50-year-old', 'dalit', 'woman', "'s", 'house', 'sets', 'ablaze', 'maharashtra', "'s", 'aurangabad', 'district']

 Class: Can't Decide

Tweet: RT @jannesoderstrom: Kristersson sparkar Ljungman efter M-SD-samarbetet i Skåne.

['rt', 'kristersson', 'sparkar', 'ljungman', 'efter', 'm-sd-samarbetet', 'skåne']

 Class: Relevant

Tweet: RT @sophmallinson: Lorene Scafaria (HUSTLERS), Lulu Wang (THE FAREWELL), Greta Gerwig (LITTLE WOMEN), Mati Diop (ATLANTICS), Marielle Helle…

['rt', 'lorene', 'scafaria', 'hustlers', 'lulu', 'wang', 'farewell', 'greta', 'gerwig', 'little', 'women', 'mati', 'diop', 'atlantics', 'marielle', 'helle…']

 Class: Can't Decide

Tweet: “Werbung“
Ich genieße es, wenn die Kugelkette unterwegs meine Labien streichelt.
🥰

https://t.co/BCO0rPgrfw

Klicke… https://t.co/Zz6l98Hp88

['“', 'werbung', '“', 

Tweet: RT @suelinwong: "I didn't realise they would just die like that, so quickly and so easily." We spoke to Salia Yang from Wuhan who lost her…

['rt', '``', "n't", 'realise', 'would', 'die', 'like', 'quickly', 'easily', "''", 'spoke', 'salia', 'yang', 'wuhan', 'lost', 'her…']

 Class: Can't Decide

Tweet: It's a new week 😇 Like the Bible says, we will pass through fire, but we will not get burnt 😇 
.
.
In another verse… https://t.co/QDoCBl807b

["'s", 'new', 'week', '😇', 'like', 'bible', 'says', 'pass', 'fire', 'get', 'burnt', '😇', 'another', 'verse…']

 Class: Can't Decide

Tweet: @Ron_Chitis @LarsHuntelaar25 @Lindenth_Bullen @rb_fans @marcello_1904 @Meer_Knappe1904 Schade hätte gerne deine Arg… https://t.co/ofUMAUBBms

['schade', 'hätte', 'gerne', 'deine', 'arg…']

 Class: Not Relevant

Tweet: Wenn's Merz wird, wirds schwarz-braun im Bund. Da kommt Freude auf.

ABER DIE LINKEN

['wenn', "'s", 'merz', 'wird', 'wirds', 'schwarz-braun', 'im', 'bund', 'da', 'kommt', 'freude', 'auf', 

Tweet: RT @InternetSpoider: Die

['rt', 'die']

 Class: Not Relevant

Tweet: RT @vanRukhoven: Geachte @WijkagentBennie. Gisteren is onze complete set tuinmeubelen weggewaaid. Wat schetst m'n verbazing; zojuist zie ik…

['rt', 'geachte', 'gisteren', 'onze', 'complete', 'set', 'tuinmeubelen', 'weggewaaid', 'wat', 'schetst', "m'n", 'verbazing', 'zojuist', 'zie', 'ik…']

 Class: Not Relevant

Tweet: @OkwasiaBiNti @Amma_Ns3mone @aparem_isaac @sey_amen @dEo_nN @therealk_okyere @wagyimikradodo @kokoBerry_ @Kayjnr10… https://t.co/oW75uqQuhv

[]

 Class: Not Relevant

Tweet: RT @fh0500357: 【MV】 CHILL FIRE / Nature's Gangstar https://t.co/3cs2FbuexB

['rt', '【mv】', 'chill', 'fire', 'nature', "'s", 'gangstar']

 Class: Can't Decide

Tweet: @El_Grande_Marv @_FriedrichMerz Das ist auch richtig. Aber die Sozialdemokraten innerhalb der CDU können halt so re… https://t.co/DNOvj0D1Y1

['das', 'ist', 'auch', 'richtig', 'aber', 'die', 'sozialdemokraten', 'innerhalb', 'der', 'cdu', 'können', 'halt', 're…'

InterfaceError: Error binding parameter 19 - probably unsupported type.